In [1]:
################
### UI SHELL ###
################

from tkinter import *
root = Tk()
root.title("Chatbot")
def send():
    send = "You -> "+e.get()
    txt.insert(END, "n"+send)
    user = e.get().lower()
    if(user == "hello"):
        txt.insert(END, "n" + "Bot -> Hi")
    elif(user == "hi" or user == "hii" or user == "hiiii"):
        txt.insert(END, "n" + "Bot -> Hello")
    elif(e.get() == "how are you"):
        txt.insert(END, "n" + "Bot -> fine! and you")
    elif(user == "fine" or user == "i am good" or user == "i am doing good"):
        txt.insert(END, "n" + "Bot -> Great! how can I help you.")
    else:
        txt.insert(END, "n" + "Bot -> Sorry! I dind't got you")
    e.delete(0, END)
txt = Text(root)
txt.grid(row=0, column=0, columnspan=2)
e = Entry(root, width=100)
e.grid(row=1, column=0)
send = Button(root, text="Send", command=send).grid(row=1, column=1)
root.mainloop()

In [22]:
###############
### IMPORTS ###
###############

import pandas as pd
import requests
from keras.preprocessing.text import Tokenizer
import re

In [13]:
#################
### Functions ###
#################

def reformat_text(text):
    text = re.sub(r'(?<=[^\s0-9])(?=[.,;?])', r' ', text)
    text = re.sub(r'\((\d+)\)', r'', text)
    text = re.sub(r'\s\s', ' ', text)
    return text

def text_processing(text):
    reformatted_text = reformat_text(text)
    tokenizer = Tokenizer(filters='"#$%&*+/:;<=>?@[\\]^_`{|}~\t\n')
    tokenizer.fit_on_texts([reformatted_text])
    s = tokenizer.texts_to_sequences([reformatted_text])[0]
    s = ' '.join(tokenizer.index_word[i] for i in s)
    return s

In [3]:
##########################
### Obtaining the Data ###
##########################
# Takes about 25 mins to run, returns 50 000 results

token = '7FDWFncezt1eTHOZvbJc78wQkgyC5QuF6Mln1NcD'
rows = 100 # fetch 100 records at a time
start = 0  # start with the first result
abstracts = [] # we'll store the bibcodes of all of our results here
titles = []

docs = True
while docs:
    # note that this URL is the same as above, except we've added parameters for start and rows
    results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?" \
                       "q=%20abs%3AThermodynamics" \
                       "&sort=date%20desc%2C%20bibcode%20desc" \
                       "&fl=title,abstract", \
                       "&rows={rows}" \
                       "&start={start}".format(rows=rows,start=start), \
                       headers={'Authorization': 'Bearer ' + token})
    try:
        docs = results.json()['response']['docs']
    except KeyError:
        print('No docs found')
        break
    # pull the bibcodes out of the results into a list
    tmp_abstract = [d['abstract'] for d in docs]
    tmp_titles = [d['title'] for d in docs]
    titles = titles + tmp_titles
    abstracts = abstracts + tmp_abstract
    start += rows # increment the start value to move to the next page of results

print(len(titles))
print(len(abstracts))


No docs found
50000
50000


In [29]:
data = {'Title':titles,'Abstract':abstracts}
data = pd.DataFrame(data)
data['Title'] = data['Title'].apply(lambda x: x[0])
column_values = data[['Title']].values.ravel()
unique_values =  pd.unique(column_values)
print(len(unique_values))

10
